In [19]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.fft import fft2, ifft2
from numpy.polynomial import chebyshev as cheb
from scipy.linalg import kron
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [20]:
# a)
x, y = [-10, 10]
n = 64
beta = 1
D_1, D_2 = 0.1, 0.1
m = 1
L = 20
N = n * n
tspan = np.arange(0, 4+0.5, 0.5)
x_uniform = np.linspace(-L / 2, L / 2, n + 1)
x = x_uniform[:n]
y = x_uniform[:n]
kx = (2 * np.pi / L) * np.hstack((np.linspace(0, n//2 - 1, n//2), np.linspace(-n//2, -1, n//2)))
ky = kx
X, Y = np.meshgrid(x, y)
KX, KY = np.meshgrid(kx, ky)
K2 = KX**2 + KY**2

def reaction_diffusion(t, uv, K2, D_1, D_2, beta, n, N):
    u = np.reshape(uv[:N], (n, n))
    v = np.reshape(uv[N:], (n, n))
    u_ifft = ifft2(u)
    v_ifft = ifft2(v)
    u3_ifft = u_ifft**3; v3_ifft = v_ifft**3
    u2v_ifft = (u_ifft**2) * v_ifft; uv2_ifft = u_ifft * (v_ifft**2)
    urhs = u_ifft - u3_ifft - uv2_ifft + beta * u2v_ifft + beta * v3_ifft
    vrhs = v_ifft - u2v_ifft - v3_ifft - beta * u3_ifft - beta * uv2_ifft
    u_diffuse = -D_1 * K2 * u
    v_diffuse = -D_2 * K2 * v
    return np.concatenate([(u_diffuse + fft2(urhs)).reshape(N), (v_diffuse + fft2(vrhs)).reshape(N)])

U = np.zeros((len(x), len(y), len(tspan))) 
V = np.zeros((len(x), len(y), len(tspan))) 

U[:, :, 0] = np.tanh(np.sqrt(X**2+Y**2))*np.cos(m*np.angle(X+1j*Y)-(np.sqrt(X**2+Y**2)))
V[:, :, 0] = np.tanh(np.sqrt(X**2+Y**2))*np.sin(m*np.angle(X+1j*Y)-(np.sqrt(X**2+Y**2)))
UV0 = np.concatenate([fft2(U[:,:,0]).flatten(),fft2(V[:,:,0]).flatten()])

UVsol = solve_ivp(reaction_diffusion, (tspan[0], tspan[-1]), y0=UV0, t_eval=tspan, args=(K2, D_1, D_2, beta, n, N))
U=UVsol.y[:N].reshape((n,n,len(tspan)))
V=UVsol.y[N:].reshape((n,n,len(tspan)))

A1 = UVsol.y
print(A1)

[[ 24.94003847+0.00000000e+00j  12.73268299+1.06593190e-15j
   -1.38095598+2.82995087e-16j ... -64.02389647-3.75173258e-14j
  -67.76356741-2.00176702e-14j -61.18058974-1.24550484e-15j]
 [-18.55666362-5.81663109e+01j -42.51586944-4.69129224e+01j
  -60.80795253-2.57480390e+01j ... -26.39439597+1.13082890e+02j
    6.86544434+1.23000456e+02j  41.4436393 +1.10055312e+02j]
 [-16.04755868+3.28279829e+01j -22.03971648-4.57977740e+01j
  -23.23089505-1.04141716e+02j ... -25.03391682-9.26527314e+01j
  -29.2936105 -4.09594873e+01j -31.3712619 +1.56986891e+01j]
 ...
 [ 24.73021466-5.66774723e+02j  34.94179045-3.31372917e+02j
   38.82924248-4.97842318e+01j ...   4.99619196+6.02396295e+02j
   -9.93322885+4.90736906e+02j -25.6299042 +2.81792021e+02j]
 [ 25.33720124-3.61633792e+02j  43.00958768-4.53711746e+02j
   51.93221654-4.47841562e+02j ... -30.76392977+2.66442187e+02j
  -58.45411318+4.29165358e+02j -74.0191717 +5.05315322e+02j]
 [ -6.4753501 +3.96245454e+01j  15.86720969-5.83358549e+01j
   37.7389

In [21]:
# b) 
x, y = [-10, 10]
n = 30
beta = 1
D_1, D_2 = 0.1, 0.1
m = 1
L = 10
N = (n + 1) * (n + 1)
tspan = np.arange(0, 4+0.5, 0.5)

# Create Chebyshev differentiation matrices
def cheb(N):
    if N == 0:
        D = 0.
        x = 1.
    else:
        n = np.arange(0, N+1)
        x = np.cos(np.pi * n / N).reshape(N+1, 1)
        c = (np.hstack(([2.], np.ones(N-1), [2.])) * (-1)**n).reshape(N+1, 1)
        X = np.tile(x, (1, N+1))
        dX = X - X.T
        D = np.dot(c, 1./c.T) / (dX + np.eye(N+1))
        D -= np.diag(np.sum(D.T, axis=0))
    return D, x.reshape(N+1)

D, x = cheb(n)
D[n, :] = 0; D[0, :] = 0
y = x
X, Y = np.meshgrid(x, y)
D2 = np.dot(D, D) / (L**2)
X = L * X
Y = L * Y
I = np.eye(len(D2))
Laplacian = kron(I, D2) + kron(D2, I)

def omega(u, v):
    return -beta * (u**2 + v**2)

def lambda_A(u, v):
    return 1 - (u**2 + v**2)

def reaction_diffusion_noflux(t, uv, D2, D_1, D_2, beta):
    u = uv[:N].reshape((n+1, n+1))
    v = uv[N:].reshape((n+1, n+1))

    u_rhs = lambda_A(u, v) * u - omega(u, v) * v
    v_rhs = omega(u, v) * u + lambda_A(u, v) * v

    u_diffuse = D_1 * Laplacian.dot(u.flatten()).reshape(n+1, n+1)
    v_diffuse = D_2 * Laplacian.dot(v.flatten()).reshape(n+1, n+1)

    return np.concatenate([(u_diffuse + u_rhs).flatten(), (v_diffuse + v_rhs).flatten()])

U = np.zeros((n+1, n+1)) 
V = np.zeros((n+1, n+1)) 
U = np.tanh(np.sqrt(X**2+Y**2))*np.cos(m*np.angle(X+1j*Y)-(np.sqrt(X**2+Y**2)))
V = np.tanh(np.sqrt(X**2+Y**2))*np.sin(m*np.angle(X+1j*Y)-(np.sqrt(X**2+Y**2)))

# Solve the system
tspan = np.arange(0, 4+0.5, 0.5)
UV0 = np.concatenate([U.flatten(), V.flatten()])

UVsol = solve_ivp(reaction_diffusion_noflux, (tspan[0], tspan[-1]), 
                  y0=UV0, t_eval=tspan, 
                  args=(D2, D_1, D_2, beta))

print(UVsol.y)
A2 = UVsol.y

[[ 0.70358468  0.27678435 -0.21775865 ... -0.79689015 -0.40972859
   0.07776933]
 [ 0.73241275  0.47188952  0.07344742 ... -0.96577657 -0.78500366
  -0.4261521 ]
 [ 0.81058026  0.37605887 -0.11123233 ... -0.84008598 -0.49565779
  -0.03085913]
 ...
 [ 0.58562756  0.91352592  0.97914313 ... -0.50294695 -0.84298442
  -0.97634716]
 [ 0.6808609   0.87018536  0.97997159 ... -0.16453512 -0.5878894
  -0.88455009]
 [ 0.71061143  0.96093661  0.97601586 ... -0.60413504 -0.91222169
  -0.99697897]]


In [24]:
# Part a - Periodic BC
def animate_periodic_UV():
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Reshape solutions from part a
    u_sol = A1[:N].reshape((n, n, len(tspan)))
    v_sol = A1[N:].reshape((n, n, len(tspan)))
    
    # Combine U and V for initial plot
    UV_combined = np.sqrt(ifft2(u_sol[:,:,0]).real**2 + ifft2(v_sol[:,:,0]).real**2)
    
    # Initial plot
    plot = ax.pcolormesh(X, Y, UV_combined, shading='auto')
    plt.colorbar(plot, ax=ax)
    ax.set_title('|U + iV| - Periodic BC')
    
    def update(frame):
        UV_combined = np.sqrt(ifft2(u_sol[:,:,frame]).real**2 + ifft2(v_sol[:,:,frame]).real**2)
        plot.set_array(UV_combined.flatten())
        return [plot]
    
    plt.tight_layout()
    anim = FuncAnimation(fig, update, frames=len(tspan), interval=200)
    anim.save('periodic_UV_animation.gif', writer='pillow')
    plt.close()
    return HTML(anim.to_jshtml())

# Part b - No-flux BC
def animate_noflux_UV():
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Reshape solutions from part b
    u_sol = A2[:N].reshape((n+1, n+1, len(tspan)))
    v_sol = A2[N:].reshape((n+1, n+1, len(tspan)))
    
    # Combine U and V for initial plot
    UV_combined = np.sqrt(u_sol[:,:,0]**2 + v_sol[:,:,0]**2)
    
    # Initial plot
    plot = ax.pcolormesh(X, Y, UV_combined, shading='auto')
    plt.colorbar(plot, ax=ax)
    ax.set_title('|U + iV| - No-flux BC')
    
    def update(frame):
        UV_combined = np.sqrt(u_sol[:,:,frame]**2 + v_sol[:,:,frame]**2)
        plot.set_array(UV_combined.flatten())
        return [plot]
    
    plt.tight_layout()
    anim = FuncAnimation(fig, update, frames=len(tspan), interval=200)
    anim.save('noflux_UV_animation.gif', writer='pillow')
    plt.close()
    return HTML(anim.to_jshtml())